In [6]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 928 µs


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
import pylab as plt
import json
import matplotlib as mpl
import os

import seaborn as sns
sns.set(style="darkgrid")

time: 1.8 ms


In [8]:
pwd

'/Users/sophieayling/Documents/GitHub/covid19-agent-based-model/notebooks/by district'

time: 2.55 ms


In [9]:
old_new_dist = pd.read_csv('../../data/raw/district_relation.csv')
old_new_dist_map = old_new_dist[['DIST2012', 'NEW_DIST_ID_2']].set_index('DIST2012').to_dict()['NEW_DIST_ID_2']
pop_dense = pd.read_csv('../../data/raw/district_pop_dens_friction.csv')
old_new_dist = old_new_dist.merge(pop_dense, on='DIST2012')

old_new_dist.head()

,Unnamed: 0,CNTRY_NAME_x,ADMIN_NAME_x,CNTRY_CODE_x,IPUM2012_x,DIST2012,PARENT_x,NEW_DIST_ID_2,CNTRY_NAME_y,ADMIN_NAME_y,CNTRY_CODE_y,IPUM2012_y,PARENT_y,mean_pop_dens,total_pop,mean_friction_pop_weighted
0,1,Zimbabwe,Bulawayo Urban,716,21,21,0,1,Zimbabwe,Bulawayo Urban,716,21,0,1096.578765,629436.210860,0.002247
1,2,Zimbabwe,Buhera,716,1001,101,1,3,Zimbabwe,Buhera,716,1001,1,40.271155,266876.945520,0.009854
2,3,Zimbabwe,Chimanimani,716,1002,102,1,4,Zimbabwe,Chimanimani,716,1002,1,36.526321,151584.230765,0.015841
3,4,Zimbabwe,Chipinge Rural,716,1003,103,1,5,Zimbabwe,Chipinge Rural,716,1003,1,51.065586,332947.622011,0.012035
4,5,Zimbabwe,Makoni,716,1004,104,1,6,Zimbabwe,Makoni,716,1004,1,31.112857,299616.813787,0.010436


time: 34.1 ms


In [11]:
import geopandas as gpd

# p = gpd.read_file('../data/geo2_zw2012.shp')
shape = p = gpd.read_file('../../data/raw/shapefiles/new_districts/ZWE_adm2.shp')

ImportError: dlopen(/Applications/anaconda3/envs/aivins_abm/lib/python3.8/site-packages/fiona/ogrext.cpython-38-darwin.so, 2): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: /Applications/anaconda3/envs/aivins_abm/lib/libgdal.27.dylib
  Reason: Incompatible library version: libgdal.27.dylib requires version 14.0.0 or later, but libjpeg.9.dylib provides version 12.0.0

time: 120 ms


In [ ]:
shape['total_population'] = shape['ID_2'].map(old_new_dist.groupby('NEW_DIST_ID_2')['total_pop'].sum())

# Trajectory by district

In [ ]:
from scenario_models import *

In [ ]:
import pickle

In [ ]:
def get_model_statistics(model, ward_id=None, scale=22):
    model_data = pd.DataFrame(
        {
            'date_infected': model.date_infected,
            'date_start_hospitalized': model.date_start_hospitalized,
            'date_start_critical': model.date_start_critical,
            'date_recovered': model.date_recovered,
            'date_died': model.date_died,
            'ward_id': list(map(lambda x: model.params.WARD_ID_TO_NAME.get(x, x), model.ward_ids)),
            'ward_id_code': model.ward_ids,
            'infected_at_ward_id': list(map(lambda x: model.params.WARD_ID_TO_NAME.get(x, x), model.infected_at_ward_ids)),
            'symptomatic_status': model.infected_symptomatic_status,
            'age': model.age,
            'economic_status_id': list(map(lambda x: model.params.ECON_STAT_ID_TO_NAME.get(x, x), model.economic_status_ids)),
            'economic_status_id_code': model.economic_status_ids,
        }
    )

    model_data['ID_2'] = model_data['ward_id'].str.strip('w_').astype(int)  # .map(old_new_dist_map)

    if ward_id is not None:
        model_data = model_data[model_data['ward_id'] == ward_id]

    return model_data


def get_active_cases(model_data, ref_col, start_col, end_cols, date_start='2020-04-20', date_end='2020-12-30', scale=22):
    hosps = model_data[model_data[start_col] < datetime.max].copy()
    hosps['day_start'] = hosps[start_col].dt.date

    ends = []
    for col in end_cols:
        ends.append(hosps[col][hosps[col] < datetime.max].dt.date)

    hosps['day_end'] = pd.concat(ends)

    ind = pd.period_range(start=date_start, end=date_end, freq='D')
    ind = ind.to_timestamp()

    col_name = start_col.split('_')[-1]
    col_name = f'active_{col_name}'

    active_hospitalizations = pd.DataFrame(columns=[col_name], index=ind).fillna(0)

    active_hosp_range_count = hosps.groupby(['day_start', 'day_end'])[ref_col].count()

    for start, end in active_hosp_range_count.index:
        active_hospitalizations.loc[start:end, col_name] += scale * active_hosp_range_count[(start, end)]

    return active_hospitalizations


def epidemic_statistics(model_data, case_col='date_infected', date_start='2020-04-20', date_end='2020-11-20', scale=22):

#     hospitalized = model_data[model_data['date_start_hospitalized'] < datetime.max]
#     q = hospitalized['date_start_hospitalized'].dt.date.value_counts().sort_index()

    hospitalized = model_data[model_data[case_col] < datetime.max].copy()
    q = hospitalized[case_col].dt.date.value_counts().sort_index()

    q = scale * q

    ind = pd.period_range(start=date_start, end=date_end, freq='D')
    ind = ind.to_timestamp()

    return q[ind].fillna(0).cumsum()

In [ ]:
scale = 11

scenario_files = {
#     'UnmitigatedScenario': 'Unmitigated w/o hand\nwashing considerations',
    'HandWashingRiskScenario': 'Unmitigated w/ hand\nwashing considerations',
#     'HandWashingRiskImproved1Scenario': 'Unmitigated w/ improved\nhand washing',
#     'HandWashingRiskImproved2Scenario': 'Unmitigated w/ extreme hand\nwashing',
#     'BlockGreatestMobilityScenario': 'Selective Blocking',
#     'LockdownGreatestMobilityScenario': 'Selective lockdown',
#     'OpenSchoolsScenario': 'Continued lockdown - open schools',
#     'OpenManufacturingScenario': 'Continued lockdown - open manufacturing',
#     'OpenMiningScenario': 'Continued lockdown - open mining',
#     'ContinuedLockdownScenario': 'Continued lockdown',
#     'OpenManufacturingAndSchoolsScenario': 'Continued lockdown - open\nmanufacturing/schools',
#     'InteractionSensitivityScenario': 'Unmitigated - homogeneous interaction',
}


for scenario in scenario_files:
    scenario_code = '_'.join(scenario.split('.')[0].split('_')[2:])
    scenario_code = scenario_files.get(scenario)
    scenario_df = []

    for sim_id, dump_file in enumerate(glob.glob(f'./logs/model_dump_file_{scenario}*R1.9*.pickle')):
        print(f'Loading {dump_file}...')

        try:
            with open(dump_file, 'rb') as fl:
                model = pickle.load(fl)
        except pickle.UnpicklingError:
            continue

        model_df = get_model_statistics(model, scale=scale)
        model_df['sim_id'] = sim_id

        scenario_df.append(model_df)

    scenario_df = pd.concat(scenario_df)

In [ ]:
len(scenario_df)

In [ ]:
date_start = '2020-04-20'
date_end = '2020-12-01'
period = 'D'
ind_start = pd.period_range(start=date_start, end=date_end, freq=period)
ind_start

In [ ]:
pd.Timedelta?

In [ ]:
date_start = '2020-04-20'
date_end = '2020-12-01'
period = 'D'
# date_end = scenario_df.date_recovered[scenario_df.date_recovered < datetime.max].max()
# date_end = date_end.strftime('%Y-%m-%d')
if period == 'D':
    ind_start = pd.period_range(start=date_start, end=date_end, freq=period)
    ind_end = ind_start + pd.Timedelta(1, unit=period)
    ind = list(zip(ind_start, ind_end))
else:
    ind = pd.period_range(start=date_start, end=date_end, freq=period)

case_col = 'date_infected'
scenario_data = []

for sim_id in scenario_df.sim_id.unique():
    print(sim_id)
    md = scenario_df[scenario_df.sim_id == sim_id].copy()

    for dist_id in md.ID_2.unique():
        dmd = md[md.ID_2==dist_id].copy()

        for e in ind:
            if period == 'D':
                st, en = e
            elif period == 'W':
                st, en = str(e).split('/')

            if dmd[dmd[case_col] < datetime.max].empty:
                i = 0
            else:
                i = epidemic_statistics(dmd, case_col=case_col, date_start=st, date_end=en, scale=scale).diff().sum()
            scenario_data.append([sim_id, dist_id, st, i])

In [ ]:
tperiod = 'week' if period == 'W' else 'day'
scenario_data_df = pd.DataFrame(scenario_data, columns=['sim_id', 'ID_2', tperiod, case_col.split('_')[-1]])
unmitigated_scenario_data_df = scenario_data_df.copy()

In [ ]:
old_new_dist.groupby('NEW_DIST_ID_2')['total_pop'].sum().head()

In [ ]:
spatial_scenario_data_df = scenario_data_df.merge(shape[['ID_2', 'NAME_2']], on='ID_2', how='left')

In [ ]:
inbound_5 = ['Harare', 'Zvimba', 'Goromonzi', 'Bulawayo', 'Masvingo']
outbound_5 = ['Harare', 'Goromonzi', 'Zvimba', 'Bulawayo', 'Masvingo']

top_5_inbound = spatial_scenario_data_df[spatial_scenario_data_df['NAME_2'].isin(inbound_5)].copy()
top_5_outbound = spatial_scenario_data_df[spatial_scenario_data_df['NAME_2'].isin(outbound_5)].copy()

In [ ]:
pp = top_5_inbound[tperiod].loc[0]
pp

In [ ]:
tstamps = top_5_inbound[tperiod].map(lambda x: x.to_timestamp())
top_5_inbound[f'{tperiod}s'] = tstamps
top_5_inbound['district'] = top_5_inbound['NAME_2']

In [ ]:
import matplotlib.dates as mdates

In [ ]:
plt.figure(figsize=(8, 5), dpi=500)

ax = sns.lineplot(x=f'{tperiod}s', y="infected", hue='district',
             data=top_5_inbound, n_boot=10)
ax.patch.set_alpha(0)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.set_title('Active cases in top 5 districts by movement')

In [ ]:
sns.lineplot(x="week", y="infected", hue='NAME_2',
             data=top_5_outbound)

In [ ]:
sns.lineplot(x="week", y="infected", hue='ID_2',
             data=scenario_data_df)

In [ ]:
ax = scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[1].plot()
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[2].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[3].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[4].plot(ax=ax)

In [ ]:
ax = scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[1].plot()
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[2].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[3].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[4].plot(ax=ax)

In [ ]:
ax = scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[1].plot()
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[2].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[3].plot(ax=ax)
scenario_data_df.groupby(['ID_2', 'week'])['infected'].mean()[4].plot(ax=ax)